In [193]:
import pyarrow.feather as feather

In [194]:
# Result is pandas.DataFrame
read_df_iot_pmfp_data = feather.read_feather('/Users/nguyentrungchinh/Documents/AITOMATIC_Test/iot_pmfp_data.feather')

In [195]:
import os

read_df_iot_pmfp_data.to_csv('/Users/nguyentrungchinh/Documents/AITOMATIC_Test/iot_pmfp_data.csv')

In [5]:
read_df_iot_pmfp_data

,datetime,machineID,volt,rotate,pressure,vibration,model,age,anomaly
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,False
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,False
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,False
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,False
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,False
...,...,...,...,...,...,...,...,...,...
876095,2016-01-01 02:00:00,100,179.438162,395.222827,102.290715,50.771941,model4,5,False
876096,2016-01-01 03:00:00,100,189.617555,446.207972,98.180607,35.123072,model4,5,False
876097,2016-01-01 04:00:00,100,192.483414,447.816524,94.132837,48.314561,model4,5,False
876098,2016-01-01 05:00:00,100,165.475310,413.771670,104.081073,44.835259,model4,5,False


In [6]:
import pandas as pd
# apply the dtype attribute
result = read_df_iot_pmfp_data.dtypes

In [7]:
result

datetime     datetime64[ns]
machineID             int64
volt                float64
rotate              float64
pressure            float64
vibration           float64
model                object
age                   int64
anomaly                bool
dtype: object

In [8]:
import numpy as np

# df_left_new_sort_datetime['failure_ID'] = np.where(df_left_new_sort_datetime['failure'] == False, 0, 1) 

read_df_iot_pmfp_data['anomaly_final'] = np.where(read_df_iot_pmfp_data['anomaly'] == False, 'There is no anomaly', 'There are some anomalies') 

read_df_iot_pmfp_data['date'] = [d.date() for d in read_df_iot_pmfp_data['datetime']]

read_df_iot_pmfp_data['time'] = [d.time() for d in read_df_iot_pmfp_data['datetime']]

read_df_iot_pmfp_data['volt_round'] = round(read_df_iot_pmfp_data['volt'], 6)

read_df_iot_pmfp_data['rotate_round'] = round(read_df_iot_pmfp_data['rotate'], 6)

read_df_iot_pmfp_data['pressure_round'] = round(read_df_iot_pmfp_data['pressure'], 6)

read_df_iot_pmfp_data['vibration_round'] = round(read_df_iot_pmfp_data['vibration'], 6)


def add_df_first_table(column_name):
    
    change_column = str(column_name) + '_rate_change'
    change_column_abs = str(column_name) + '_rate_change_abs'
    change_column_type = str(column_name) + '_rate_change_type'

    read_df_iot_pmfp_data[str(change_column)] = (read_df_iot_pmfp_data[str(column_name)] - read_df_iot_pmfp_data[str(column_name)].shift(1))/read_df_iot_pmfp_data[str(column_name)].shift(1)
    read_df_iot_pmfp_data[str(change_column_type)] = read_df_iot_pmfp_data[str(change_column)].apply(lambda x: 'increase' if x > 0 else 'decrease' if x<0 else 'not chance' if x==0 else pd.NA)
    read_df_iot_pmfp_data[str(change_column_abs)] = abs(read_df_iot_pmfp_data[str(change_column)])
    read_df_iot_pmfp_data[str(change_column_abs)] = round(read_df_iot_pmfp_data[str(change_column_abs)]*100, 4)
    
    column_lag1 = str(column_name) + '_lag1'
    change_column_lag1 = str(column_name) + '_rate_change_lag1'
    change_column_abs_lag1 = str(column_name) + '_rate_change_abs_lag1'
    change_column_type_lag1 = str(column_name) + '_rate_change_type_lag1'
    
    read_df_iot_pmfp_data[str(column_lag1)] = read_df_iot_pmfp_data[str(column_name)].shift(1)
    read_df_iot_pmfp_data[str(change_column_lag1)] = read_df_iot_pmfp_data[str(change_column)].shift(1)
    read_df_iot_pmfp_data[str(change_column_type_lag1)] = read_df_iot_pmfp_data[str(change_column_type)].shift(1)
    read_df_iot_pmfp_data[str(change_column_abs_lag1)] = read_df_iot_pmfp_data[str(change_column_abs)].shift(1)
    
add_df_first_table('rotate')

add_df_first_table('volt')

add_df_first_table('pressure')

add_df_first_table('vibration')

read_df_iot_pmfp_data['machineID_lag1'] = read_df_iot_pmfp_data[str('machineID')].shift(1)

read_df_iot_pmfp_data['model_lag1'] = read_df_iot_pmfp_data[str('model')].shift(1)

read_df_iot_pmfp_data['age_lag1'] = read_df_iot_pmfp_data[str('age')].shift(1)

read_df_iot_pmfp_data['description'] = 'At at the time ' + read_df_iot_pmfp_data['time'].astype(str) + ' in ' + read_df_iot_pmfp_data['date'].astype(str) \
+ ', the machine which has machine ID ' + read_df_iot_pmfp_data['machineID'].astype(str)  + ' and model ID ' + read_df_iot_pmfp_data['model'].astype(str) + ' and ' + read_df_iot_pmfp_data['age'].astype(str) + ' age ' \
+ ', has hourly average voltage ' + read_df_iot_pmfp_data['volt_rate_change_type'].astype(str) + ' about ' + read_df_iot_pmfp_data['volt_rate_change_abs'].astype(str) + ' percent ' \
+ ' and hourly average rotation ' + read_df_iot_pmfp_data['rotate_rate_change_type'].astype(str) + ' about ' + read_df_iot_pmfp_data['rotate_rate_change_abs'].astype(str) + ' percent ' \
+ ' and hourly average pressure ' + read_df_iot_pmfp_data['pressure_rate_change_type'].astype(str) + ' about ' + read_df_iot_pmfp_data['pressure_rate_change_abs'].astype(str) + ' percent ' \
+ ' and hourly average vibration ' + read_df_iot_pmfp_data['vibration_rate_change_type'].astype(str) + ' about ' + read_df_iot_pmfp_data['vibration_rate_change_abs'].astype(str) + ' percent .'

read_df_iot_pmfp_data['description_absolute'] = 'At the time ' + read_df_iot_pmfp_data['time'].astype(str) + ' in ' + read_df_iot_pmfp_data['date'].astype(str) \
+ ', the machine which has machine ID ' + read_df_iot_pmfp_data['machineID'].astype(str)  + ' and model ID ' + read_df_iot_pmfp_data['model'].astype(str) + ' and ' + read_df_iot_pmfp_data['age'].astype(str) + ' age ' \
+ ', has hourly average voltage ' + read_df_iot_pmfp_data['volt_round'].astype(str) \
+ ' and hourly average rotation ' + read_df_iot_pmfp_data['rotate_round'].astype(str) \
+ ' and hourly average pressure ' + read_df_iot_pmfp_data['pressure_round'].astype(str) \
+ ' and hourly average vibration ' + read_df_iot_pmfp_data['vibration_round'].astype(str) + ' .'

read_df_iot_pmfp_data['description_lag1'] = read_df_iot_pmfp_data['description'].shift(1)
read_df_iot_pmfp_data['description_absolute_lag1'] = read_df_iot_pmfp_data['description_absolute'].shift(1)


In [9]:
read_df_iot_pmfp_labels = feather.read_feather('/Users/nguyentrungchinh/Documents/AITOMATIC_Test/iot_pmfp_labels.feather')

In [10]:
read_df_iot_pmfp_labels

,datetime,machineID,failure_comp1,failure_comp2,failure_comp3,failure_comp4,maint_comp1,maint_comp2,maint_comp3,maint_comp4,error1,error2,error3,error4,error5,failure,maint,error,anomaly
0,2015-01-01 06:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
1,2015-01-01 07:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
2,2015-01-01 08:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
3,2015-01-01 09:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
4,2015-01-01 10:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876095,2016-01-01 02:00:00,100,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
876096,2016-01-01 03:00:00,100,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
876097,2016-01-01 04:00:00,100,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
876098,2016-01-01 05:00:00,100,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False


In [11]:
# apply the dtype attribute
result2 = read_df_iot_pmfp_labels.dtypes

In [12]:
result2

datetime         datetime64[ns]
machineID                 int64
failure_comp1             uint8
failure_comp2             uint8
failure_comp3             uint8
failure_comp4             uint8
maint_comp1               uint8
maint_comp2               uint8
maint_comp3               uint8
maint_comp4               uint8
error1                    uint8
error2                    uint8
error3                    uint8
error4                    uint8
error5                    uint8
failure                    bool
maint                      bool
error                      bool
anomaly                    bool
dtype: object

In [13]:
# sample data based on conditions

read_df_iot_pmfp_labels_resampling_true_v2 = read_df_iot_pmfp_labels[read_df_iot_pmfp_labels['failure'] == True]

read_df_iot_pmfp_labels_resampling_false_v2 = read_df_iot_pmfp_labels[read_df_iot_pmfp_labels['failure'] == False]

read_df_iot_pmfp_labels_resampling_false_subsample_v2 = read_df_iot_pmfp_labels_resampling_false_v2.sample(n = 800)

read_df_iot_pmfp_labels_resampling_false_subsample_v2

read_df_iot_pmfp_labels_resampling_concat_v2 = pd.concat([read_df_iot_pmfp_labels_resampling_true_v2, read_df_iot_pmfp_labels_resampling_false_subsample_v2])

read_df_iot_pmfp_labels_resampling_concat_v2

,datetime,machineID,failure_comp1,failure_comp2,failure_comp3,failure_comp4,maint_comp1,maint_comp2,maint_comp3,maint_comp4,error1,error2,error3,error4,error5,failure,maint,error,anomaly
96,2015-01-05 06:00:00,1,0,0,0,1,1,0,0,1,0,0,0,0,0,True,True,False,True
1536,2015-03-06 06:00:00,1,1,0,0,0,1,0,0,0,0,0,0,0,0,True,True,False,True
2616,2015-04-20 06:00:00,1,0,1,0,0,0,1,0,0,0,0,0,0,0,True,True,False,True
4056,2015-06-19 06:00:00,1,0,0,0,1,1,0,0,1,0,0,0,0,0,True,True,False,True
5856,2015-09-02 06:00:00,1,0,0,0,1,1,0,0,1,0,0,0,0,0,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456832,2015-02-22 18:00:00,53,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
105582,2015-01-20 00:00:00,13,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
661444,2015-07-02 07:00:00,76,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
221744,2015-04-24 13:00:00,26,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False


In [14]:
import pandas as pd

df_left_new = pd.merge(read_df_iot_pmfp_data, read_df_iot_pmfp_labels_resampling_concat_v2, on=['datetime', 'machineID', 'anomaly'], how='inner')

df_left_new

,datetime,machineID,volt,rotate,pressure,vibration,model,age,anomaly,anomaly_final,...,maint_comp3,maint_comp4,error1,error2,error3,error4,error5,failure,maint,error
0,2015-01-05 06:00:00,1,179.303153,499.777962,111.833028,52.383097,model3,18,True,There are some anomalies,...,0,1,0,0,0,0,0,True,True,False
1,2015-01-21 11:00:00,1,160.968427,437.988743,98.090919,46.541963,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,0,False,False,False
2,2015-02-07 01:00:00,1,172.199380,433.293145,105.438246,31.416984,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,0,False,False,False
3,2015-03-06 06:00:00,1,198.257975,456.862342,89.333995,38.671900,model3,18,True,There are some anomalies,...,0,0,0,0,0,0,0,True,True,False
4,2015-03-20 22:00:00,1,182.503299,445.509696,90.552131,40.674257,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,2015-10-05 07:00:00,100,177.337094,427.697488,108.270165,42.909691,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,0,False,False,False
1515,2015-12-02 07:00:00,100,162.128959,403.825018,111.815220,41.846011,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,0,False,False,False
1516,2015-12-09 06:00:00,100,144.720190,377.188361,109.995936,28.433400,model4,5,True,There are some anomalies,...,0,0,0,0,0,0,0,True,True,False
1517,2015-12-15 14:00:00,100,167.932947,534.694395,83.218797,42.686264,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,0,False,False,False


In [15]:
df_left_new['failure_ID'] = np.where(df_left_new['failure'] == False, 0, 1) 

In [16]:
df_left_new

,datetime,machineID,volt,rotate,pressure,vibration,model,age,anomaly,anomaly_final,...,maint_comp4,error1,error2,error3,error4,error5,failure,maint,error,failure_ID
0,2015-01-05 06:00:00,1,179.303153,499.777962,111.833028,52.383097,model3,18,True,There are some anomalies,...,1,0,0,0,0,0,True,True,False,1
1,2015-01-21 11:00:00,1,160.968427,437.988743,98.090919,46.541963,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
2,2015-02-07 01:00:00,1,172.199380,433.293145,105.438246,31.416984,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
3,2015-03-06 06:00:00,1,198.257975,456.862342,89.333995,38.671900,model3,18,True,There are some anomalies,...,0,0,0,0,0,0,True,True,False,1
4,2015-03-20 22:00:00,1,182.503299,445.509696,90.552131,40.674257,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,2015-10-05 07:00:00,100,177.337094,427.697488,108.270165,42.909691,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
1515,2015-12-02 07:00:00,100,162.128959,403.825018,111.815220,41.846011,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
1516,2015-12-09 06:00:00,100,144.720190,377.188361,109.995936,28.433400,model4,5,True,There are some anomalies,...,0,0,0,0,0,0,True,True,False,1
1517,2015-12-15 14:00:00,100,167.932947,534.694395,83.218797,42.686264,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0


In [17]:
df_left_new.dtypes

datetime      datetime64[ns]
machineID              int64
volt                 float64
rotate               float64
pressure             float64
                   ...      
error5                 uint8
failure                 bool
maint                   bool
error                   bool
failure_ID             int64
Length: 68, dtype: object

In [18]:
df_left_new_final = df_left_new.iloc[1:]

In [19]:
df_left_new_final

,datetime,machineID,volt,rotate,pressure,vibration,model,age,anomaly,anomaly_final,...,maint_comp4,error1,error2,error3,error4,error5,failure,maint,error,failure_ID
1,2015-01-21 11:00:00,1,160.968427,437.988743,98.090919,46.541963,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
2,2015-02-07 01:00:00,1,172.199380,433.293145,105.438246,31.416984,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
3,2015-03-06 06:00:00,1,198.257975,456.862342,89.333995,38.671900,model3,18,True,There are some anomalies,...,0,0,0,0,0,0,True,True,False,1
4,2015-03-20 22:00:00,1,182.503299,445.509696,90.552131,40.674257,model3,18,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
5,2015-04-20 06:00:00,1,180.050801,346.362480,105.661164,39.218055,model3,18,True,There are some anomalies,...,0,0,0,0,0,0,True,True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,2015-10-05 07:00:00,100,177.337094,427.697488,108.270165,42.909691,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
1515,2015-12-02 07:00:00,100,162.128959,403.825018,111.815220,41.846011,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
1516,2015-12-09 06:00:00,100,144.720190,377.188361,109.995936,28.433400,model4,5,True,There are some anomalies,...,0,0,0,0,0,0,True,True,False,1
1517,2015-12-15 14:00:00,100,167.932947,534.694395,83.218797,42.686264,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0


In [20]:
df_left_new_final_sort_by_datetime = df_left_new_final.sort_values(by=['datetime'])

In [21]:
df_left_new_final_sort_by_datetime

,datetime,machineID,volt,rotate,pressure,vibration,model,age,anomaly,anomaly_final,...,maint_comp4,error1,error2,error3,error4,error5,failure,maint,error,failure_ID
740,2015-01-02 03:00:00,51,195.899676,286.393505,79.226021,44.595823,model4,19,True,There are some anomalies,...,0,0,0,0,0,0,True,False,False,1
206,2015-01-02 03:00:00,16,170.524081,405.302970,109.855730,37.786365,model1,3,True,There are some anomalies,...,0,0,0,0,0,0,True,False,False,1
223,2015-01-02 03:00:00,17,158.098828,406.325391,97.001765,59.967174,model1,14,True,There are some anomalies,...,0,0,0,0,0,0,True,False,False,1
304,2015-01-02 03:00:00,22,204.623416,426.794399,119.485200,44.055151,model1,14,True,There are some anomalies,...,0,0,0,0,0,0,True,False,False,1
489,2015-01-02 03:00:00,35,174.495255,508.561648,90.875311,37.334185,model1,17,True,There are some anomalies,...,0,0,0,0,0,0,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,2015-12-31 11:00:00,16,145.708197,442.021300,118.113865,47.081500,model1,3,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
1134,2015-12-31 13:00:00,77,186.371801,483.278159,109.518415,32.642681,model4,12,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
116,2015-12-31 15:00:00,9,169.037919,490.834420,92.099912,44.012965,model4,7,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0
1518,2015-12-31 19:00:00,100,138.506098,497.861210,112.525039,34.867196,model4,5,False,There is no anomaly,...,0,0,0,0,0,0,False,False,False,0


In [39]:
import os

path = "/Users/nguyentrungchinh/Documents/AITOMATIC_Test/logs_new_onnx/test_2/"

df_left_new_final_sort_by_datetime.to_csv(os.path.join(path + 'df_left_new_final_sort_by_datetime.csv'))

In [1]:
import pandas as pd
import os

path = "/Users/nguyentrungchinh/Documents/AITOMATIC_Test/logs_new_onnx/test_2/"

df_final = pd.read_csv(os.path.join(path + 'df_left_new_final_sort_by_datetime.csv'))

In [2]:
df_final

,Unnamed: 0,datetime,machineID,volt,rotate,pressure,vibration,model,age,anomaly,...,maint_comp4,error1,error2,error3,error4,error5,failure,maint,error,failure_ID
0,617,2015-01-01 21:00:00,43,189.707332,518.138645,104.342546,45.525591,model3,14,False,...,0,0,0,0,0,0,False,False,False,0
1,104,2015-01-02 00:00:00,9,156.893222,443.630812,109.145101,31.102902,model4,7,False,...,0,0,0,0,0,0,False,False,False,0
2,827,2015-01-02 03:00:00,58,167.187100,331.460659,106.277021,38.808612,model1,5,True,...,0,0,0,0,0,0,True,False,False,1
3,1155,2015-01-02 03:00:00,80,166.557579,419.798170,80.990671,45.920842,model3,6,True,...,0,0,0,0,0,0,True,False,False,1
4,1138,2015-01-02 03:00:00,79,206.934554,423.353299,110.745533,35.708642,model3,14,True,...,0,0,0,0,0,0,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,203,2015-12-31 06:00:00,15,189.340234,420.190682,106.031828,46.339284,model3,14,True,...,1,0,0,0,0,0,True,True,False,1
1514,929,2015-12-31 06:00:00,64,190.820355,495.747474,105.052513,47.237444,model3,20,True,...,0,0,0,0,0,0,True,True,False,1
1515,1354,2015-12-31 06:00:00,90,215.728683,467.771439,100.961884,44.469377,model2,2,True,...,0,0,0,0,0,0,True,True,False,1
1516,1437,2015-12-31 06:00:00,95,184.969808,507.191449,122.983617,50.224889,model2,18,True,...,0,0,0,0,0,0,True,True,False,1


In [3]:
import numpy as np

values_test, counts_test = np.unique(df_final['failure_ID'], return_counts=True)

In [4]:
values_test

array([0, 1])

In [5]:
counts_test

array([800, 718])

In [6]:
train_percent = round(len(df_final)*0.8)
train_df_v2 = df_final.iloc[0:train_percent]
validation_df_v2 = df_final.iloc[train_percent:]
print('TRAIN DATA is ~80% :', len(train_df_v2))
print('VALIDATION DATA is ~20% :', len(validation_df_v2))

TRAIN DATA is ~80% : 1214
VALIDATION DATA is ~20% : 304


In [7]:
train_df_v2.rename(columns={"failure_ID": "label"},inplace = True)
validation_df_v2.rename(columns={"failure_ID": "label"}, inplace = True)

# train_df_v2.rename(columns={"failure": "label"},inplace = True)
# validation_df_v2.rename(columns={"failure": "label"}, inplace = True)

/var/folders/vg/kwrjbctx5tbfzxswtdv503jw0000gn/T/ipykernel_50138/2159732687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_v2.rename(columns={"failure_ID": "label"},inplace = True)
/var/folders/vg/kwrjbctx5tbfzxswtdv503jw0000gn/T/ipykernel_50138/2159732687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_df_v2.rename(columns={"failure_ID": "label"}, inplace = True)


In [8]:
train_df_v2.rename(columns={"description_absolute_lag1": "text"},inplace = True)
validation_df_v2.rename(columns={"description_absolute_lag1": "text"}, inplace = True)

/var/folders/vg/kwrjbctx5tbfzxswtdv503jw0000gn/T/ipykernel_50138/98703783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_v2.rename(columns={"description_absolute_lag1": "text"},inplace = True)
/var/folders/vg/kwrjbctx5tbfzxswtdv503jw0000gn/T/ipykernel_50138/98703783.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_df_v2.rename(columns={"description_absolute_lag1": "text"}, inplace = True)


In [9]:
train_df_v2_revise = train_df_v2[['label','text']]
validation_df_v2_revise = validation_df_v2[['label','text']]

In [10]:
train_df_v2_revise

,label,text
0,0,"At the time 20:00:00 in 2015-01-01, the machin..."
1,0,"At the time 23:00:00 in 2015-01-01, the machin..."
2,1,"At the time 02:00:00 in 2015-01-02, the machin..."
3,1,"At the time 02:00:00 in 2015-01-02, the machin..."
4,1,"At the time 02:00:00 in 2015-01-02, the machin..."
...,...,...
1209,0,"At the time 05:00:00 in 2015-10-11, the machin..."
1210,1,"At the time 05:00:00 in 2015-10-12, the machin..."
1211,1,"At the time 05:00:00 in 2015-10-12, the machin..."
1212,1,"At the time 05:00:00 in 2015-10-12, the machin..."


In [11]:
train_df_v2_revise.dtypes

label     int64
text     object
dtype: object

In [12]:
import datasets
from datasets import Dataset, DatasetDict

In [13]:
train_tds = Dataset.from_pandas(train_df_v2_revise)
val_tds = Dataset.from_pandas(validation_df_v2_revise)

In [16]:
dd_pm = datasets.DatasetDict({"train":train_tds,"val":val_tds})
dd_pm

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1214
    })
    val: Dataset({
        features: ['label', 'text'],
        num_rows: 304
    })
})

In [36]:
model_name = 'microsoft/xtremedistil-l6-h384-uncased'

In [37]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
tokenized_dd_pm = dd_pm.map(tokenize_function, batched=True)

Map:   0%|          | 0/1214 [00:00<?, ? examples/s]

Map:   0%|          | 0/304 [00:00<?, ? examples/s]

In [38]:
tokenized_dd_pm

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1214
    })
    val: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 304
    })
})

In [39]:
tokenized_dd_pm['train'].features

{'label': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
tokenized_dd_pm['train']['label']

In [41]:
train_dd_pm = tokenized_dd_pm["train"]
eval_dd_pm = tokenized_dd_pm["val"]

In [26]:
!pip3 install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [42]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [43]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/var/folders/vg/kwrjbctx5tbfzxswtdv503jw0000gn/T/ipykernel_50138/722645183.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [45]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="/Users/nguyentrungchinh/Documents/AITOMATIC_Test/logs_new_onnx/test_trainer",
                                  per_device_train_batch_size=128, 
                                  num_train_epochs=24,learning_rate=3e-05,
                                  evaluation_strategy="epoch")
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dd_pm,
    eval_dataset=eval_dd_pm,
    compute_metrics=compute_metrics,
)

In [168]:
!pip3 install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [46]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.698967,0.496711
2,No log,0.692896,0.496711
3,No log,0.618468,0.513158
4,No log,0.395677,0.976974
5,No log,0.270386,0.976974
6,No log,0.201413,0.976974
7,No log,0.167017,0.976974
8,No log,0.148220,0.976974
9,No log,0.136924,0.976974
10,No log,0.130049,0.976974


TrainOutput(global_step=240, training_loss=0.2639360745747884, metrics={'train_runtime': 170.4648, 'train_samples_per_second': 170.921, 'train_steps_per_second': 1.408, 'total_flos': 241579757592576.0, 'train_loss': 0.2639360745747884, 'epoch': 24.0})

In [47]:
trainer.evaluate()

{'eval_loss': 0.11405101418495178,
 'eval_accuracy': 0.9769736842105263,
 'eval_runtime': 0.8947,
 'eval_samples_per_second': 339.767,
 'eval_steps_per_second': 42.471,
 'epoch': 24.0}

In [48]:
import transformers
import transformers.convert_graph_to_onnx as onnx_convert
from pathlib import Path

In [49]:
pipeline = transformers.pipeline("text-classification",model=model,tokenizer=tokenizer)

In [50]:
model = model.to("cpu")

In [27]:
!pip3 install onnx onnxruntime 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Obtaining dependency information for onnx from https://files.pythonhosted.org/packages/19/94/99e19571a9cda6852ac54ddd4a6ff6645e69b09c7faa415894d034dae125/onnx-1.14.1-cp311-cp311-macosx_10_12_universal2.whl.metadata
  Obtaining dependency information for onnxruntime from https://files.pythonhosted.org/packages/6b/4a/fda588a7a296fa0a5b1618d7de7a1be583125b7f6616463618a092c613de/onnxruntime-1.16.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Obtaining dependency information for flatbuffers from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuff

In [70]:
onnx_convert.convert_pytorch(pipeline, opset=11, output=Path("/Users/nguyentrungchinh/Documents/AITOMATIC_Test/logs_new_onnx/Output_model_onnx/classifier_3.onnx"), use_external_format=False)

Using framework PyTorch: 2.1.0
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


In [71]:
from onnxruntime.quantization import quantize_dynamic, QuantType
quantize_dynamic("/Users/nguyentrungchinh/Documents/AITOMATIC_Test/logs_new_onnx/Output_model_onnx/classifier_3.onnx", "/Users/nguyentrungchinh/Documents/AITOMATIC_Test/logs_new_onnx/Output_model_quantization/classifier_3_int8.onnx", 
                 weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.5/atten

In [72]:
import onnxruntime as ort

In [73]:
session = ort.InferenceSession("/Users/nguyentrungchinh/Documents/AITOMATIC_Test/logs_new_onnx/Output_model_onnx/classifier_3.onnx")
session_int8 = ort.InferenceSession("/Users/nguyentrungchinh/Documents/AITOMATIC_Test/logs_new_onnx/Output_model_quantization/classifier_3_int8.onnx")

In [80]:
import numpy
import matplotlib

In [81]:
import scipy

In [75]:
input_feed = {
    "input_ids": np.array(eval_dd_pm['input_ids']),
    "attention_mask": np.array(eval_dd_pm['attention_mask']),
    "token_type_ids": np.array(eval_dd_pm['token_type_ids'])
}

In [76]:
out = session.run(input_feed=input_feed,output_names=['output_0'])[0]
out_int8 = session_int8.run(input_feed=input_feed,output_names=['output_0'])[0]

In [87]:
import numpy as np
from scipy.special import softmax

probs_out = softmax(out, axis= -1)
probs_out_int8 = softmax(out_int8, axis= -1)

In [77]:
predictions = np.argmax(out, axis=-1)
predictions_int8 = np.argmax(out_int8, axis=-1)

In [78]:
metric.compute(predictions=predictions, references=eval_dd_pm['label'])

{'accuracy': 0.9769736842105263}

In [82]:
metric.compute(predictions=predictions_int8, references=eval_dd_pm['label'])

{'accuracy': 0.9769736842105263}

In [48]:
!pip3 install cmake

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
!pip3 install wheel setuptools --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
!pip3 install ez_setup

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [51]:
!pip3 install --upgrade pip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [52]:
!pip3 install --upgrade setuptools

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [55]:
!pip3 install --upgrade scipy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [56]:
!pip3 install --upgrade setuptools

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [60]:
!pip3 install matplotlib --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/af/f3/fb27b3b902fc759bbca3f9d0336c48069c3022e57552c4b0095d997c7ea8/matplotlib-3.8.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 3.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.2
    Uninstalling matplotlib-3.7.2:
      Successfully uninstalled matplotlib-3.7.2
